# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686503


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:44,  1.60s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:54,  2.02s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:52,  2.03s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:37,  1.51s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:15<00:00,  3.27it/s]

Rendering models:  93%|█████████▎| 28/30 [00:15<00:00,  3.45it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  3.17it/s]

Rendering models: 100%|██████████| 30/30 [00:16<00:00,  3.09it/s]

benjaminthomas                        0.000684
CThomas                               0.000595
not-logged-in-f7e2260ed4a9517a3e4a    0.022872
Andy_Carleen                          0.000204
not-logged-in-742cef5e1a82d2ddd0fb    0.000189
Camelron                              0.000192
Dynamatt                              0.000212
austingasbarra                        0.000197
thedarktree                           0.000172
not-logged-in-9a45911eb71184fe7c7f    0.000559
not-logged-in-5f30b2bb9b7f9d49da41    1.170172
not-logged-in-1a34827709c37b568bba    0.013128
Samuel_Liuwe                          0.000821
not-logged-in-9e3f2b1e338eb5a2154c    0.000330
AlexYoung35                           0.000189
gabrielweiss                          0.002049
acapirala                             0.000220
surfgimp                              0.000420
Lavadude                              0.021624
not-logged-in-8700ff8251e0704f745c    0.000152
briansav7                             0.001045
ISlicedI     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())